In [ ]:
import os
import sys
import glob
import argparse
import matplotlib.pyplot as plt

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.models import load_model

IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
NB_EPOCHS = 3
BAT_SIZE = 32
FC_SIZE = 2

def get_nb_files(directory):
    """Get number of files by searching directory recursively"""
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt


def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet

    Args:
        base_model: keras model excluding top
        nb_classes: # of classes

    Returns:
        new keras model with last layer
    """
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
    predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
    model = Model(input=base_model.input, output=predictions)
    return model


def setup_to_finetune(model):
    first_trainable_layer = -1
    for layer in model.layers:
        if not layer.trainable:
            first_trainable_layer+=1
        else:
            break
    first_trainable_layer -= 10
    for layer in model.layers[:first_trainable_layer]:
         layer.trainable = False
    for layer in model.layers[first_trainable_layer:]:
         layer.trainable = True
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    return first_trainable_layer

def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    plt.figure()
    plt.plot(epochs, loss, 'r.')
    plt.plot(epochs, val_loss, 'r-')
    plt.title('Training and validation loss')
    plt.show()


train_dir = "./input/train/"
val_dir = "./input/dev/"

nb_epoch = NB_EPOCHS
batch_size = BAT_SIZE

nb_train_samples = get_nb_files(train_dir)
nb_classes = len(glob.glob(train_dir + "/*"))
nb_val_samples = get_nb_files(val_dir)
nb_epoch = int(nb_epoch)
batch_size = int(batch_size)

# data prep
train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
)
test_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=batch_size,
)

validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=batch_size,
)

# setup model
base_model = InceptionV3(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
model = add_new_last_layer(base_model, nb_classes)

# transfer learning
setup_to_transfer_learn(model, base_model)

history_tl = model.fit_generator(
    train_generator,
    nb_epoch=nb_epoch,
    validation_steps = nb_val_samples//batch_size,
    steps_per_epoch=nb_train_samples//batch_size,
    validation_data=validation_generator,
    class_weight='auto')

model.save("./model/inceptionv3-tl.model")

In [ ]:
from keras.models import load_model
model = load_model("./model/inceptionv3-tl.model")

# fine-tuning
for i in range(30):
    j = setup_to_finetune(model)
    print("Unfreeze layer:", j)

    history_ft = model.fit_generator(
        train_generator,
        nb_epoch=1,
        steps_per_epoch=nb_train_samples//batch_size,
        validation_steps = nb_val_samples//batch_size,
        validation_data=validation_generator,
        class_weight='auto')

    model.save("./model/inceptionv3-ft" + str(j) + ".model")

In [ ]:
history_ft = model.fit_generator(
    train_generator,
    nb_epoch=1,
    steps_per_epoch=nb_train_samples//batch_size,
    validation_steps = nb_val_samples//batch_size,
    validation_data=validation_generator,
    class_weight='auto')

model.save("./model/inceptionv3-final.model")

In [ ]:
from keras.models import load_model
from tqdm import tqdm 
import numpy as np
from PIL import Image
from os import listdir
from os.path import isfile, join
from keras.preprocessing import image as im
from keras.applications.inception_v3 import InceptionV3, preprocess_input
import matplotlib.pyplot as plt

model = load_model("./model/inceptionv3-final.model")

target_size = (229, 229) 

def predict(model, img, target_size):
    if img.size != target_size:
        img = img.resize(target_size)
    preds = []
    for i in [img, np.fliplr(img)]:
        x = im.img_to_array(i)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        preds.append(model.predict(x)[0])
    if np.max(preds[0])>np.max(preds[1]):
        return preds[0]
    else:
        return preds[1]

testpath = "./input/test/not/"
images = [(testpath+f) for f in listdir(testpath) if isfile(join(testpath, f))]
cnt = 0
for image in tqdm(images):
    img = Image.open(image)
    preds = predict(model, img, target_size)
    cnt += preds[0]>0.5

print(cnt/len(images), cnt, len(images))
#0.954299571 5116 5361

testpath = "./input/test/sun/"
images = [(testpath+f) for f in listdir(testpath) if isfile(join(testpath, f))]
cnt = 0
for image in tqdm(images):
    #print(image)
    img = Image.open(image)
    preds = predict(model, img, target_size)
    cnt += preds[0]<0.5
    
print(cnt/len(images), cnt, len(images))
#0.918421052632 2094 2280

In [ ]:
(2094+5116)/(2280+5361)
#0.9435937704488941